<a href="https://colab.research.google.com/github/fiaazwa/chatbot-rag-papua/blob/main/Cerita_rakyat_papua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-community pypdf tiktoken

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install --upgrade --quiet langchain langchain-community pypdf tiktoken

In [4]:
!pip install chromadb


In [8]:
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

pdf_paths = [
    "/content/drive/MyDrive/cerita-rakyat-papua/dataset.pdf/10-Cerita-Rakyat-Papua-Terpilih-2012.pdf",
    "/content/drive/MyDrive/cerita-rakyat-papua/dataset.pdf/Kumpulan-Cerita-Rakyat-Papua.pdf",
    "/content/drive/MyDrive/cerita-rakyat-papua/dataset.pdf/Cerita-dari-Tanah-Papua.pdf"
]

def bersihkan_teks(teks):
    teks = re.sub(r'[^\x00-\x7F]+', ' ', teks)
    teks = re.sub(r'\s+', ' ', teks)
    return teks.strip()

def load_filter_clean_pdf(path):
    loader = PyPDFLoader(path)
    pages = loader.load()
    cleaned_pages = []
    for p in pages:
        content = p.page_content.strip()
        if len(content) > 100:
            cleaned = bersihkan_teks(content)
            cleaned_pages.append(cleaned)
    return cleaned_pages

all_clean_texts = []
for path in pdf_paths:
    if os.path.exists(path):
        all_clean_texts.extend(load_filter_clean_pdf(path))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.create_documents(all_clean_texts)


In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Load model embedding ringan
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Buat vector store dari dokumen yang sudah di-split
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory="rag_db"  # kamu bisa ganti path ini sesuai keinginan
)

# Simpan vectorstore ke disk
vectorstore.persist()

print("Embedding selesai dan vectorstore berhasil disimpan.")


Embedding selesai dan vectorstore berhasil disimpan.


In [10]:
!unzip "/content/drive/MyDrive/cerita-rakyat-papua/rag_db.zip" -d "/content/drive/MyDrive/cerita-rakyat-papua/"

Archive:  /content/drive/MyDrive/cerita-rakyat-papua/rag_db.zip
   creating: /content/drive/MyDrive/cerita-rakyat-papua/rag_db/


In [12]:
!pip install pyngrok

In [17]:
!ngrok config add-authtoken 2xaByQ7RNGxVLwn1Jlw1UdtfYpb_3kSi9MgMZr3T7Y9wJ1vRY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [19]:
from pyngrok import ngrok
public_url = ngrok.connect(addr="8501", proto="http")
print("🔗 Streamlit app URL:", public_url)


🔗 Streamlit app URL: NgrokTunnel: "https://4553-35-245-9-44.ngrok-free.app" -> "http://localhost:8501"


In [20]:
!streamlit run /content/drive/MyDrive/cerita-rakyat-papua/app.py --server.port 8501 &>/content/logs.txt &